In [ ]:
import sys
sys.path.insert(1, 'utils')
from utils import *
from data_manipulation import *
pd.options.display.max_colwidth = 100

In [ ]:
dataset = pd.read_csv('data/training_data.csv')

### Un vistazo a las features

In [ ]:
dataset.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.model_selection import train_test_split

### Un poco de limpieza

In [ ]:
dataset['warrant_time'] = dataset.warrant_time.fillna(0)
dataset['seller_status'] = dataset.seller_status.fillna('N/A')

dataset.dropna(subset=['initial_quantity', 'created_date_diff_decile', 'last_update_diff_decile'], inplace=True)
dataset.reset_index(drop=True, inplace=True)

#### One Hot Encoding para las Variables categorícas

In [ ]:
seller_status_columns = ["seller_status_{}".format(idx) for idx,_val in enumerate(dataset.seller_status.unique())]
pub_condition_columns = ["pub_condition_{}".format(idx) for idx,_val in enumerate(dataset.condition.unique())]

In [ ]:
one_hot_encoder = OneHotEncoder(sparse=False)
ohc = one_hot_encoder.fit_transform(dataset['seller_status'].values.reshape(-1,1))
dataset = pd.concat([dataset, pd.DataFrame(ohc, columns=seller_status_columns)], axis=1)

In [ ]:
one_hot_encoder = OneHotEncoder(sparse=False, )
ohc = one_hot_encoder.fit_transform(dataset['condition'].values.reshape(-1,1))
dataset = pd.concat([dataset, pd.DataFrame(ohc, columns=pub_condition_columns)], axis=1).reindex(dataset.index)

## __Selección de features__

In [ ]:
features = ['price', 'available_quantity', 'initial_quantity', 'discount_per', 'installment_num', 'installment_rate', 
            'pic_qty', 'created_date_diff', 'created_date_diff_decile','last_update_diff_decile', 'has_free_shipping',
            'last_update_date_diff', 'warrant_time', 'is_official', 'has_video', 'qty_variations', 'qty_attributes'] +  seller_status_columns + pub_condition_columns

predictors = dataset[features]
predictors.isnull().any()

In [ ]:
predictors.describe()

## __Test Train Split__

### Se tomará 20% de los datos para validación

In [ ]:
target = dataset['sold_quantity']
x_train, x_cv, y_train, y_cv = train_test_split(predictors, target, test_size=0.2, random_state=1)

# __Modelos__

### Cómo se trata de un problema de regresión donde tengo el target que quiero predecir, voy a utilizar modelos de tipo arbol y regresiones lineales pues estos generalmente dan buenos resultados inclusive con poca cantidad de features o la poca cantidad de datos utilizados, los modelos a comparar serán:
* Gradient Boosting
* Random Forest
* Lasso (Regresión lineal con regularización Lasso)
* KNN

### Cómo medida de performance, voy a usar __R²__ para medir la performance del modelo, dado que este valor permite conocer que tanta dispersión hay en la variable independiente respecto de las variables independientes. Para comparar distintos modelos ademas voy a usar el __RMSE__. 

# Gradient Boosting

### Voy a probar múltiples parámetros para encontrar el óptimo, solo con gradient boosting, pues es el que mejores resultados presento en comparación con los demás

In [ ]:
rmse_models = []

In [ ]:
params = [{'max_depth': [3,10,20], 'learning_rate': [0.0001, 0.01, 0.1, 0.5], 'n_estimators': [100, 150, 200, 250]}]

gb = GridSearchCV(GradientBoostingRegressor(), params, cv=3)
gb.fit(x_train, y_train)
y_pred_gb = gb.predict(x_cv)
gb_r2 = r2_score(y_cv, y_pred_gb)
gb_mse = mean_squared_error(y_cv, y_pred_gb)
gb_rmse = math.sqrt(gb_mse)
rmse_models.append(['Gradient Boost', gb_rmse])
print("R²: {}\nMSE: {}\nRMSE: {}".format(gb_r2, gb_mse, gb_rmse ))

## Los parámetros elegidos son:

In [ ]:
gb.best_params_

# Random Forest

In [ ]:
rf = RandomForestRegressor(n_estimators=200)
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_cv)
rf_r2 = r2_score(y_cv, y_pred_rf)
rf_mse = mean_squared_error(y_cv, y_pred_rf)
rf_rmse = math.sqrt(rf_mse)
rmse_models.append(['Random Forest', rf_rmse])
print("R²: {}\nMSE: {}\nRMSE: {}".format(rf_r2, rf_mse, rf_rmse ))

# Lasso

In [ ]:
lso = RandomForestRegressor(n_estimators=200)
lso.fit(x_train, y_train)
y_pred_lso = lso.predict(x_cv)
lso_r2 = r2_score(y_cv, y_pred_lso)
lso_mse = mean_squared_error(y_cv, y_pred_lso)
lso_rmse = math.sqrt(lso_mse)
rmse_models.append(['Lasso', lso_rmse])
print("R²: {}\nMSE: {}\nRMSE: {}".format(lso_r2, lso_mse, lso_rmse ))

# KNN

In [ ]:
knn = KNeighborsRegressor()
knn.fit(x_train, y_train)
y_pred_knn = knn.predict(x_cv)
knn_r2 = r2_score(y_cv, y_pred_knn)
knn_mse = mean_squared_error(y_cv, y_pred_knn)
knn_rmse = math.sqrt(knn_mse)
rmse_models.append(['KNN', knn_rmse])
print("R²: {}\nMSE: {}\nRMSE: {}".format(knn_r2, knn_mse, knn_rmse ))

## __Resultados__

In [ ]:
test_set = pd.concat([x_cv, y_cv], axis=1)
test_set['y_pred'] = y_pred_gb
test_set = test_set.join(pd.DataFrame(dataset.category), how='left')
test_set_by_cat = test_set.groupby('category')

r2_by_cat = []
for cat, test in test_set_by_cat:
    r2_by_cat.append([cat,math.sqrt(mean_squared_error(test.sold_quantity, test.y_pred))])
r2_by_cat = pd.DataFrame(r2_by_cat, columns=['category', 'error'])

In [ ]:
plt.figure(figsize = (15, 6))
sns.set_theme(style="whitegrid")
ax = sns.barplot(x="category", y="error", data=r2_by_cat)
ax.set_title("RMSE por cateogría")
ax.set(xlabel='Category', ylabel='R² Score')

### Viendo el error cuadrático medio para las distintas categorías se puede ver que el modelo comete muchos errores al predecir la cantidad vendida de televisores en comparación con las otras categorías

## __Comparando Modelos__

In [ ]:
rmse_mod = pd.DataFrame(rmse_models, columns=['model', 'RMSE'])

plt.figure(figsize = (15, 6))
ax = sns.barplot(data=rmse_mod, x="model", y="RMSE")
ax.set_title("RMSE por modelo")
ax.set(xlabel='Category', ylabel='RMSE')

## Gráfico de los residuos para algunas features

#### Calculo de residuo estandarizado

In [ ]:
test_set['residual'] = test_set.sold_quantity - test_set.y_pred
mean_residual = test_set.residual.mean()
std_residual = test_set.residual.std()
test_set['standard_residual'] = (test_set.residual - mean_residual)/std_residual

In [ ]:
plt.figure(figsize = (15, 6))
ax = sns.residplot(data=test_set, x='price', y='standard_residual')
ax.set_title("Standarized residual plot")
ax.set(xlabel='Price', ylabel='Residuo estandarizado')

In [ ]:
plt.figure(figsize = (15, 6))
ax = sns.residplot(data=test_set, x='initial_quantity', y='standard_residual')
ax.set_title("Standarized residual plot")
ax.set(xlabel='initial_quantity', ylabel='Residuo estandarizado')

In [ ]:
plt.figure(figsize = (15, 6))
ax = sns.residplot(data=test_set, x='is_official', y='standard_residual')
ax.set_title("Standarized residual plot")
ax.set(xlabel='is_official', ylabel='Residuo estandarizado')

### __Feature Importances Gradient Boost__

### Para ver un poco como se comportaron las features del Gradient Boost, calculo el feature importance y estas podrán apreciarse en la siguiente visualización

In [ ]:
importances = list(gb.best_estimator_.feature_importances_)

In [ ]:
fig = plt.figure(figsize=(25, 5))
ax = fig.add_subplot(111)
x_values = list(range(len(importances)))
ax.bar(x_values, importances, orientation = 'vertical')
plt.xticks(x_values, features, rotation='vertical', fontsize=12)
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Feature Importances');
